Load Events

In [1]:
import pandas as pd
import numpy as np

In [2]:
events = pd.read_csv('events.csv')

Event Mean Regression Creation

In [3]:
events['Event Date'] = pd.to_datetime(events['Event Date'])
events['Today'] = pd.to_datetime("now")
events['Time to Event'] = events['Event Date'] - events['Today']
events['Time to Event Number'] = pd.to_numeric(events['Time to Event'])/(1000000000*60*60*24)

In [4]:
events['Event Mean Regression'] = np.log10(events['Time to Event Number'])

In [5]:
events

,Event Name,Event ID,Event Type,EventLength,Event Date,Today,Time to Event,Time to Event Number,Event Mean Regression
0,LVC,47877,Elite,4,2021-02-25,2021-02-24 04:29:00.236782,0 days 19:30:59.763218,0.813192,-0.089807
1,Waco,48685,Elite,3,2021-03-12,2021-02-24 04:29:00.236782,15 days 19:30:59.763218,15.813192,1.199020
2,Open at Belton,47888,Silver,3,2021-03-19,2021-02-24 04:29:00.236782,22 days 19:30:59.763218,22.813192,1.358186
3,Vintage Open,48119,Silver,3,2021-04-08,2021-02-24 04:29:00.236782,42 days 19:30:59.763218,42.813192,1.631578
4,Jonesboro,48567,Elite,3,2021-04-16,2021-02-24 04:29:00.236782,50 days 19:30:59.763218,50.813192,1.705976
5,Mid-America,48121,Silver,3,2021-04-23,2021-02-24 04:29:00.236782,57 days 19:30:59.763218,57.813192,1.762027
6,Goat Hill,48283,Silver,3,2021-05-07,2021-02-24 04:29:00.236782,71 days 19:30:59.763218,71.813192,1.856204
7,OTBO,48172,Elite,3,2021-05-14,2021-02-24 04:29:00.236782,78 days 19:30:59.763218,78.813192,1.896599
8,Portland,48686,Elite,3,2021-06-04,2021-02-24 04:29:00.236782,99 days 19:30:59.763218,99.813192,1.999188
9,Clash at the Canyons,48112,Silver,3,2021-07-02,2021-02-24 04:29:00.236782,127 days 19:30:59.763218,127.813192,2.106576


Load Players

In [6]:
eventplayers = pd.read_csv('eventplayers.csv')

In [7]:
eventplayers

,Event,Event ID,Name,PDGA#,Rating
0,LVC,47877,Paige Pierce,29190,991
1,LVC,47877,Sarah Hokom,34563,967
2,LVC,47877,Catrina Allen,44184,962
3,LVC,47877,Hailey King,81351,962
4,LVC,47877,Valerie Mandujano,62879,956
...,...,...,...,...,...
313,PCS Sula Open,47446,Kona Star Panis,27832,926
314,PCS Sula Open,47446,Paige Pierce,29190,991
315,PCS Sula Open,47446,Rebecca Cox,32917,929
316,PCS Sula Open,47446,Sarah Hokom,34563,967


Load Scores

****Blank until rounds have been recorded but i should probably write code for it anyway

Generate Average Rating At Each Event

In [8]:
eventplayers['Rating'] = pd.to_numeric(eventplayers['Rating'])
means = eventplayers.groupby('Event ID')['Rating'].mean()
events = pd.merge(events,means,on = 'Event ID',how = 'left')

In [9]:
events

,Event Name,Event ID,Event Type,EventLength,Event Date,Today,Time to Event,Time to Event Number,Event Mean Regression,Rating
0,LVC,47877,Elite,4,2021-02-25,2021-02-24 04:29:00.236782,0 days 19:30:59.763218,0.813192,-0.089807,897.688889
1,Waco,48685,Elite,3,2021-03-12,2021-02-24 04:29:00.236782,15 days 19:30:59.763218,15.813192,1.199020,929.269231
2,Open at Belton,47888,Silver,3,2021-03-19,2021-02-24 04:29:00.236782,22 days 19:30:59.763218,22.813192,1.358186,915.931034
3,Vintage Open,48119,Silver,3,2021-04-08,2021-02-24 04:29:00.236782,42 days 19:30:59.763218,42.813192,1.631578,922.850000
4,Jonesboro,48567,Elite,3,2021-04-16,2021-02-24 04:29:00.236782,50 days 19:30:59.763218,50.813192,1.705976,930.928571
5,Mid-America,48121,Silver,3,2021-04-23,2021-02-24 04:29:00.236782,57 days 19:30:59.763218,57.813192,1.762027,906.866667
6,Goat Hill,48283,Silver,3,2021-05-07,2021-02-24 04:29:00.236782,71 days 19:30:59.763218,71.813192,1.856204,912.736842
7,OTBO,48172,Elite,3,2021-05-14,2021-02-24 04:29:00.236782,78 days 19:30:59.763218,78.813192,1.896599,917.896552
8,Portland,48686,Elite,3,2021-06-04,2021-02-24 04:29:00.236782,99 days 19:30:59.763218,99.813192,1.999188,910.454545
9,Clash at the Canyons,48112,Silver,3,2021-07-02,2021-02-24 04:29:00.236782,127 days 19:30:59.763218,127.813192,2.106576,914.058824


Actually Calculate Round Evs?

In [10]:
rating_points_per_stroke = 6

In [11]:
eventplayerratings = pd.merge(events,eventplayers, on = 'Event ID',how = 'right')
eventplayerratings['Single Round Expected Score'] = -1*((eventplayerratings['Rating_y']-eventplayerratings['Rating_x'])/(rating_points_per_stroke+eventplayerratings['Event Mean Regression']))

In [12]:
eventplayerratings

,Event Name,Event ID,Event Type,EventLength,Event Date,Today,Time to Event,Time to Event Number,Event Mean Regression,Rating_x,Event,Name,PDGA#,Rating_y,Single Round Expected Score
0,LVC,47877,Elite,4,2021-02-25,2021-02-24 04:29:00.236782,0 days 19:30:59.763218,0.813192,-0.089807,897.688889,LVC,Paige Pierce,29190,991,-15.788167
1,LVC,47877,Elite,4,2021-02-25,2021-02-24 04:29:00.236782,0 days 19:30:59.763218,0.813192,-0.089807,897.688889,LVC,Sarah Hokom,34563,967,-11.727386
2,LVC,47877,Elite,4,2021-02-25,2021-02-24 04:29:00.236782,0 days 19:30:59.763218,0.813192,-0.089807,897.688889,LVC,Catrina Allen,44184,962,-10.881389
3,LVC,47877,Elite,4,2021-02-25,2021-02-24 04:29:00.236782,0 days 19:30:59.763218,0.813192,-0.089807,897.688889,LVC,Hailey King,81351,962,-10.881389
4,LVC,47877,Elite,4,2021-02-25,2021-02-24 04:29:00.236782,0 days 19:30:59.763218,0.813192,-0.089807,897.688889,LVC,Valerie Mandujano,62879,956,-9.866194
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
313,PCS Sula Open,47446,Elite,4,2021-07-07,2021-02-24 04:29:00.236782,132 days 19:30:59.763218,132.813192,2.123241,952.200000,PCS Sula Open,Kona Star Panis,27832,926,3.225314
314,PCS Sula Open,47446,Elite,4,2021-07-07,2021-02-24 04:29:00.236782,132 days 19:30:59.763218,132.813192,2.123241,952.200000,PCS Sula Open,Paige Pierce,29190,991,-4.776419
315,PCS Sula Open,47446,Elite,4,2021-07-07,2021-02-24 04:29:00.236782,132 days 19:30:59.763218,132.813192,2.123241,952.200000,PCS Sula Open,Rebecca Cox,32917,929,2.856003
316,PCS Sula Open,47446,Elite,4,2021-07-07,2021-02-24 04:29:00.236782,132 days 19:30:59.763218,132.813192,2.123241,952.200000,PCS Sula Open,Sarah Hokom,34563,967,-1.821933
